In [1]:
#!/usr/bin/env python
# coding: utf-8

import pandas as pd
import pandas_gbq
from pykrx import stock
from pykrx import bond
import FinanceDataReader as fdr


from time import sleep

import psycopg2 as pg2
from sqlalchemy import create_engine

from datetime import datetime
from datetime import timedelta

import os
import time

import glob
from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud import storage


# 경로 변경
os.chdir('/home/shjj08choi/finance_mlops')


# 서비스 계정 키 JSON 파일 경로
key_path = glob.glob("key_value/*.json")[0]

# Credentials 객체 생성
credentials = service_account.Credentials.from_service_account_file(key_path)

# 빅쿼리 정보
project_id = 'owenchoi-396200'
dataset_id = 'finance_mlops'

# GCP 클라이언트 객체 생성
storage_client = storage.Client(credentials = credentials, 
                         project = credentials.project_id)
bucket_name = 'finance-mlops-1'     # 서비스 계정 생성한 bucket 이름 입력

# Postgresql 연결
db_connect_info = pd.read_csv('key_value/db_connect_info.csv')
username = db_connect_info['username'][0]
password = db_connect_info['password'][0]
host = db_connect_info['host'][0]
database = db_connect_info['database'][0]
engine = create_engine(f'postgresql+psycopg2://{username}:{password}@{host}:5432/{database}')



now = datetime.now()
# now = now + timedelta(days=-2)
today_date1 = now.strftime('%Y%m%d')
start_date1 = '20180101'
today_date2 = now.strftime('%Y-%m-%d')
today_date_time_csv = now.strftime("%Y%m%d_%H%M")


In [24]:
ticker_nm = '005930'

In [29]:

index_code_master = pd.read_csv('data_crawler/index_code_master/index_code_master_20230825.csv', dtype = {'ticker':object})
kor_stock_fundamental = pd.read_csv("data_crawler/kor_stock_fundamental/kor_stock_fundamental_20230922.csv")

kor_index_ohlcv = pd.read_csv("data_crawler/kor_index_ohlcv/kor_index_ohlcv_20230825.csv")



In [26]:
kor_stock_fundamental_total = kor_stock_fundamental[kor_stock_fundamental['ticker'] == ticker_nm].reset_index()


In [27]:
kor_stock_fundamental_total['bps'][0]

57822

In [28]:
index_code_master[index_code_master['ticker'] == ticker_nm]

,ticker,index_code,index_code_nm,market,corp_name
0,005930,1001,코스피,KOSPI,삼성전자
832,005930,1002,코스피 대형주,KOSPI,삼성전자
1996,005930,1013,전기전자,KOSPI,삼성전자
2492,005930,1027,제조업,KOSPI,삼성전자
2984,005930,1028,코스피 200,KOSPI,삼성전자
3184,005930,1034,코스피 100,KOSPI,삼성전자
3284,005930,1035,코스피 50,KOSPI,삼성전자
3404,005930,1155,코스피 200 정보기술,KOSPI,삼성전자
3833,005930,1224,코스피 200 비중상한 30%,KOSPI,삼성전자
4033,005930,1227,코스피 200 비중상한 25%,KOSPI,삼성전자


In [23]:
df_raw = stock.get_market_ohlcv(today_date1,  market="ALL")

In [31]:
kor_index_ohlcv[kor_index_ohlcv['index_code'] == 1013]

,date,open,high,low,close,volume,trading_value,market_cap,index_code
16728,2018-01-02,18903.39,18947.42,18773.97,18867.79,28044390,1066140547189,495995795764139,1013
16729,2018-01-03,19198.78,19213.21,19007.14,19060.52,38497478,1330818643873,500546385701954,1013
16730,2018-01-04,19262.60,19262.70,18809.88,18860.74,58105730,1474482495153,495683506981220,1013
16731,2018-01-05,18993.73,19304.78,18969.88,19304.32,41641063,1592418453669,506885866710854,1013
16732,2018-01-08,19439.35,19464.62,19057.15,19225.32,60770016,1635539536859,504762378109565,1013
...,...,...,...,...,...,...,...,...,...
18117,2023-08-21,26244.80,26376.93,26092.52,26184.03,38846995,2037066360530,806034827835709,1013
18118,2023-08-22,26543.67,26661.13,26255.31,26359.88,44899310,2445349852793,810970424703978,1013
18119,2023-08-23,26303.63,26416.77,26097.00,26246.13,70347835,2083931002188,807993645635378,1013
18120,2023-08-24,26747.88,26881.09,26671.30,26847.61,67034840,3634984037501,826205449181109,1013


In [25]:
df_raw = stock.get_market_ohlcv('20180101', '20230822', '095570')
df_raw

,시가,고가,저가,종가,거래량,등락률
날짜,,,,,,
2018-01-02,6980,6990,6850,6970,60294,0.000000
2018-01-03,6970,7000,6910,6920,45848,-0.717360
2018-01-04,6990,7100,6920,7070,53176,2.167630
2018-01-05,7080,7300,7040,7250,58573,2.545969
2018-01-08,7330,7330,7100,7130,47104,-1.655172
...,...,...,...,...,...,...
2023-08-16,4200,4270,4170,4185,85067,-3.460208
2023-08-17,4120,4230,4030,4210,118070,0.597372
2023-08-18,4200,4285,4105,4230,78260,0.475059


In [15]:
df_raw = stock.get_market_ohlcv('20180101', '20230822', '095570')
df_raw = df_raw.reset_index()
# df_raw['날짜'] = today_date2
df_raw['티커'] = ticker_nm
df_raw = df_raw[['날짜', '시가', '고가', '저가', '종가', '거래량','티커']]
df_raw.columns = ['date', 'open', 'high', 'low', 'close', 'volume',  'ticker']

df_raw['date'] = pd.to_datetime(df_raw['date'])

file_name = 'kor_stock_ohlcv'

now1 = datetime.now()
time_line = now1.strftime("%Y%m%d_%H:%M:%S")

In [28]:
df_raw = stock.get_market_cap('20180101', '20230822', '095570')
stock.get_market_cap('20180101', '20230822', ticker_nm)

,시가총액,거래량,거래대금,상장주식수
날짜,,,,
2018-01-02,326351396150,60294,417947300,46822295
2018-01-03,324010281400,45848,319107800,46822295
2018-01-04,331033625650,53176,372980010,46822295
2018-01-05,339461638750,58573,420815340,46822295
2018-01-08,333842963350,47104,338693000,46822295
...,...,...,...,...
2023-08-16,195951304575,85067,357310815,46822295
2023-08-17,197121861950,118070,485064050,46822295
2023-08-18,198058307850,78260,327369725,46822295


In [30]:
df_raw = stock.get_market_cap(today_date1,  market="ALL")
df_raw

,종가,시가총액,거래량,거래대금,상장주식수
티커,,,,,
005930,67100,400572409105000,9515526,635953731300,5969782550
373220,525000,122850000000000,190310,100645387000,234000000
000660,116000,84448274340000,2564626,295650361900,728002365
207940,763000,54305762000000,32290,24698625000,71174000
005490,551000,46598747730000,741248,407883425000,84571230
...,...,...,...,...,...
140660,1210,3009574920,0,0,2487252
245450,1598,2524776080,22,34696,1579960
288490,55,2379685275,139635,7674710,43267005


In [ ]:

# 주가 정보
print('주가정보 시작')
for ticker_nm in kor_ticker_list:
    try:
        df_raw = stock.get_market_ohlcv('20180101', '20230822', ticker_nm)
        df_raw = df_raw.reset_index()
        # df_raw['날짜'] = today_date2
        df_raw['티커'] = ticker_nm
        df_raw = df_raw[['날짜', '시가', '고가', '저가', '종가', '거래량', '거래대금', '등락률', '티커']]
        df_raw.columns = ['date', 'open', 'high', 'low', 'close', 'volume', 'trading_value', 'price_change_percentage', 'ticker']
        
        df_raw['date'] = pd.to_datetime(df_raw['date'])
        
        file_name = 'kor_stock_ohlcv'
        
        now1 = datetime.now()
        time_line = now1.strftime("%Y%m%d_%H:%M:%S")
        
        upload_df(df_raw, file_name, project_id, dataset_id, time_line, today_date1)
        print(f'주가정보 완료_{ticker_nm}_{time_line}')
    except:
        print(f'주가정보 실패_{ticker_nm}_{time_line}')